# Introduction to SyMPC

Want to discover SyMPC? You've come just to the right place! This is a short notebook to help you starting with SyMPC.

Authors: 
- George Muraru - [Twitter](https://twitter.com/GeorgeMuraru) - [GitHub](https://github.com/gmuraru)
- Théo Ryffel - [Twitter](http://twitter.com/theoryffel) - [GitHub](https://github.com/LaRiffle)

SyMPC is a library to perform efficient Secure Multi-Party Computation. It works in combination with PySyft to allow combining a wide range of privacy-preserving techniques.

Therefore, we need to import `syft`, to declare the virtual machines emulating our parties

In [1]:
import syft as sy
sy.logger.remove()
import torch

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

The `MPCTensor` is the tensor that holds reference to the shares owned by the different parties. The `Session` is a key object that stores many information about the current computation, including a reference to the parties involved!

In [2]:
# Define the virtual machines that would be use in the computation
alice_vm = sy.VirtualMachine(name="alice")
bob_vm = sy.VirtualMachine(name="bob")
charlie_vm = sy.VirtualMachine(name="charlie")

# Get clients from each VM
alice = alice_vm.get_client()
bob = bob_vm.get_client()
charlie = charlie_vm.get_client()

parties = [alice, bob, charlie]

# Setup the session for the computation
session = Session(parties=parties)
SessionManager.setup_mpc(session)

### Secret sharing

Ok we are ready to start! Let's define some PyTorch tensors.

In [3]:
# Define the private values to shares
x_secret = torch.Tensor([[0.1, -1], [-4, 4]])
y_secret = torch.Tensor([[4.0, -2.5], [5, 2]])

print(x_secret)
print(y_secret)

tensor([[ 0.1000, -1.0000],
        [-4.0000,  4.0000]])
tensor([[ 4.0000, -2.5000],
        [ 5.0000,  2.0000]])


We then build and distribute secret shares to the parties using the following syntax:

In [4]:
# Share the secret between our parties
x = x_secret.share(session=session)
y = y_secret.share(session=session)

print(x)
print(y)

[MPCTensor]
Shape: torch.Size([2, 2])
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer
	| <VirtualMachineClient: charlie Client> -> ShareTensorPointer
[MPCTensor]
Shape: torch.Size([2, 2])
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer
	| <VirtualMachineClient: charlie Client> -> ShareTensorPointer


You can print a `MPCTensor` to have useful information about it, even if its value is private.

### Addition

Addition on secret shared values is straight forward:

In [5]:
z = x + y 

print(z.reconstruct())

tensor([[ 4.1000, -3.5000],
        [ 1.0000,  6.0000]])


`.reconstruct()` get back the shares from all the parties and reconstruct the underlying value.

### Matrix Multiplication

Multiplication or matrix multiplication is also very easy! It relies on the SPDZ protocol, if you are interested to learn more about it, check our [Private AI Series](https://courses.openmined.org/)!

In [6]:
matmul_result = x @ y 

print(matmul_result.reconstruct())

tensor([[-4.6000, -2.2500],
        [ 4.0000, 18.0000]])


### What's next?

SyMPC is still under development! We will add here more features as soon they are stable enough, stay tuned! 🕺

If you enjoyed this tutorial, show your support by [Starring SyMPC](https://github.com/OpenMined/SyMPC)! 🙏